In [1]:
import numpy as np
import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
import seaborn as sns
import pandas as pd
import cv2
#image argumentation
import tensorflow as tf
import glob, os
from sklearn.model_selection import train_test_split
#설계도를 그림으로 시각화
from tensorflow.keras.utils import plot_model

In [19]:
import tensorflow as tf
from keras.layers import Activation, Convolution2D, Dropout, Conv2D
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import SeparableConv2D
from keras import layers
from keras.regularizers import l2

#설계도를 그림으로 시각화
from tensorflow.keras.utils import plot_model

In [2]:
#MODEL-littleVGG16
model_littleVGG = tf.keras.Sequential(
      [ #block-1
       tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),padding="same",kernel_initializer="he_normal",input_shape=(48,48,1),activation="relu" ),
       tf.keras.layers.BatchNormalization(),
       tf.keras.layers.Activation("relu"),
       tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),padding="same",kernel_initializer="he_normal",activation="relu" ),
       tf.keras.layers.BatchNormalization(),
       tf.keras.layers.Activation("relu"),
       tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
       tf.keras.layers.Dropout(0.2),

       #block-2
       tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),padding="same",kernel_initializer="he_normal",activation="elu" ),
       tf.keras.layers.BatchNormalization(),
       tf.keras.layers.Activation("relu"),
       tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),padding="same",kernel_initializer="he_normal",activation="elu" ),
       tf.keras.layers.BatchNormalization(),
       tf.keras.layers.Activation("relu"),
       tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
       tf.keras.layers.Dropout(0.2),

       #block-3
       tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),padding="same",kernel_initializer="he_normal",activation="elu" ),
       tf.keras.layers.BatchNormalization(),
       tf.keras.layers.Activation("relu"),
       tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),padding="same",kernel_initializer="he_normal",activation="elu" ),
       tf.keras.layers.BatchNormalization(),
       tf.keras.layers.Activation("relu"),
       tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
       tf.keras.layers.Dropout(0.2),

       #block-4
       tf.keras.layers.Conv2D(filters=256,kernel_size=(3,3),padding="same",kernel_initializer="he_normal",activation="elu" ),
       tf.keras.layers.BatchNormalization(),
       tf.keras.layers.Activation("relu"),
       tf.keras.layers.Conv2D(filters=256,kernel_size=(3,3),padding="same",kernel_initializer="he_normal",activation="elu" ),
       tf.keras.layers.BatchNormalization(),
       tf.keras.layers.Activation("relu"),
       tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
       tf.keras.layers.Dropout(0.2),

       #block-5
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(units=64, kernel_initializer='he_normal',activation="elu"),
       tf.keras.layers.BatchNormalization(),
       tf.keras.layers.Dropout(0.5),


       #blcok-6
       tf.keras.layers.Dense(units=64, kernel_initializer='he_normal',activation='elu'),
       tf.keras.layers.BatchNormalization(),
       tf.keras.layers.Dropout(0.5),

       tf.keras.layers.Dense(units=4,kernel_initializer='he_normal',activation='softmax')
       
                                                                                        
  ])
print(model_littleVGG.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0

In [24]:
def big_XCEPTION(input_shape, num_classes):
    img_input = Input((48,48,1))
    x = Conv2D(32, (3, 3), strides=(2, 2), use_bias=False)(img_input)   #레이어1 Conv2D
    x = BatchNormalization(name='block1_conv1_bn')(x)
    x = Activation('relu', name='block1_conv1_act')(x)
    x = Conv2D(64, (3, 3), use_bias=False)(x)
    x = BatchNormalization(name='block1_conv2_bn')(x)
    x = Activation('relu', name='block1_conv2_act')(x)

    residual = Conv2D(128, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization(name='block2_sepconv1_bn')(x)
    x = Activation('relu', name='block2_sepconv2_act')(x)
    x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization(name='block2_sepconv2_bn')(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    residual = Conv2D(256, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = Activation('relu', name='block3_sepconv1_act')(x)
    x = SeparableConv2D(256, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization(name='block3_sepconv1_bn')(x)
    x = Activation('relu', name='block3_sepconv2_act')(x)
    x = SeparableConv2D(256, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization(name='block3_sepconv2_bn')(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])
    x = Conv2D(num_classes, (3, 3),
            #kernel_regularizer=regularization,
            padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    output = Activation('softmax',name='predictions')(x)

    model = Model(img_input, output)
    return model

In [28]:
def XCEPTION(input_shape, num_classes):
    img_input = Input(input_shape)
    #========================Entry flow===================================

    x = Conv2D(32, (3, 3), strides=(2, 2), use_bias=False)(img_input)   #레이어1 Conv2D
    x = BatchNormalization(name='block1_conv1_bn')(x)
    x = Activation('relu', name='block1_conv1_act')(x)
    x = Conv2D(64, (3, 3), use_bias=False)(x)
    x = BatchNormalization(name='block1_conv2_bn')(x)
    x = Activation('relu', name='block1_conv2_act')(x)

    residual = Conv2D(128, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization(name='block2_sepconv1_bn')(x)
    x = Activation('relu', name='block2_sepconv2_act')(x)
    x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization(name='block2_sepconv2_bn')(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    residual = Conv2D(256, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = Activation('relu', name='block3_sepconv1_act')(x)
    x = SeparableConv2D(256, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization(name='block3_sepconv1_bn')(x)
    x = Activation('relu', name='block3_sepconv2_act')(x)
    x = SeparableConv2D(256, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization(name='block3_sepconv2_bn')(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    #추가된내용
    residual = Conv2D(728, (1,1), strides=(2,2), padding='same')(x)
    residual = BatchNormalization()(residual)

    x = Activation('relu',name='block4_sepconv1_act')(x)
    x = SeparableConv2D(728, (3,3), padding='same', use_bias=False)(x)
    x = BatchNormalization(name = 'block4_sepconv1_bn')(x)
    x = Activation('relu',name='block4_sepconv2_act')(x)
    x = SeparableConv2D(728, (3,3), padding ='same', use_bias=False)(x)
    x = BatchNormalization(name='block4_sepconv2_bn')(x)

    x = MaxPooling2D((3,3),strides=(2,2),padding='same')(x)
    x = layers.add([x, residual])


    #====================MiDDLE FlOW ==========================

    for i in range(8):
      block_prefix = 'blocks_' +(str(i+5))

      residual = x
      x = Activation('relu',name=block_prefix+'_sepconv1_act')(x)
      x = SeparableConv2D(728, (3,3), padding='same', use_bias=False)(x)
      x = BatchNormalization(name = block_prefix+'_sepconv1_bn')(x)
      x = Activation('relu',name=block_prefix+'_sepconv2_act')(x)
      x = SeparableConv2D(728, (3,3), padding='same', use_bias=False)(x)
      x = BatchNormalization(name = block_prefix+'_sepconv2_bn')(x)
      x = Activation('relu',name=block_prefix+'_sepconv3_act')(x)
      x = SeparableConv2D(728, (3,3), padding='same', use_bias=False)(x)
      x = BatchNormalization(name = block_prefix+'_sepconv3_bn')(x)

      x = layers.add([x,residual])
    
    #======================EXIT FLOW ===============================

    residual = Conv2D(1024, (1,1), strides=(2,2), padding="same")(x)
    residual = BatchNormalization()(residual)

    x= Activation('relu',name='block5_sepconv1_act')(x)
    x = SeparableConv2D(728, (3,3), padding='same', use_bias=False)(x)
    x = BatchNormalization(name = 'block5_sepconv1_bn')(x)
    x = Activation('relu',name='block5_sepconv2_act')(x)
    x = SeparableConv2D(1024, (3,3), padding='same', use_bias=False)(x)
    x = BatchNormalization(name = 'block5_sepconv2_bn')(x)
    
    x = MaxPooling2D((3,3),strides=(2,2),padding='same')(x)
    x = layers.add([x, residual])

    x = SeparableConv2D(1536, (3,3), padding='same', use_bias=False)(x)
    x = BatchNormalization(name = 'block6_sepconv1_bn')(x)
    x = Activation('relu',name='block6_sepconv1_act')(x)

    x = SeparableConv2D(2048, (3,3), padding='same', use_bias=False)(x)
    x = BatchNormalization(name = 'block6_sepconv2_bn')(x)
    x = Activation('relu',name='block6_sepconv2_act')(x)

    




    x = Conv2D(num_classes, (3, 3),#kernel_regularizer=regularization,
            padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    output = Activation('softmax',name='predictions')(x)

    model = Model(img_input, output)
    return model


                              
                              


SyntaxError: non-default argument follows default argument (<ipython-input-28-3baf041fd889>, line 1)

In [26]:
big_XCEPTION(4).summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 48, 48, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 23, 23, 32)   288         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 23, 23, 32)   128         conv2d_16[0][0]                  
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 23, 23, 32)   0           block1_conv1_bn[0][0]            
_______________________________________________________________________________________

In [32]:
XCEPTION((48,48,1),4).summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 48, 48, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 23, 23, 32)   288         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 23, 23, 32)   128         conv2d_21[0][0]                  
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 23, 23, 32)   0           block1_conv1_bn[0][0]            
_______________________________________________________________________________________